# Introduction

This notebook is a quick introduction to "pythrahyper_net", a computational network growth simulation. After loading all necessary libraries, the starting conditions and computation grid are initialized. Finally, a small simulation of a network growing on the surface of a sphere is performed.

First, we import numpy and some other python modules:

In [ ]:
import numpy as np
from time import time as timet
import concurrent.futures
import os
import itertools
import multiprocessing
from pathlib import Path

Now we need to import the main simulation classes as well as some auxiliary modules for configuration and plotting:

In [ ]:
# Import the framework's main module
import Initialize_Network_Growing as ing

# This python file loads the parameter setting for the growth simulation
import Start_Conditions.Parameter_Importer as Parameter_Importer

# This python file initializes the starting nodes and growth cones: 
import Start_Conditions.Node_Maker as Node_Maker

# This loads the images containing the structure of teh environment:
import Start_Conditions.StructField_Maker as StructField_Maker

# Growth step functionality
import Math_and_Simulation.Step_Maker as Step_Maker

# Functions for plotting and making movies
import Plots_and_Movies.Plot_Maker as Plot_Maker
import Plots_and_Movies.Movie_Maker as Movie_Maker

# Functions for generating the networkx graph
import Network_Graph_Construction.Network_X as Network_X
import Network_Graph_Construction.Update_Network as Update_Network

Set the location of the configuration files (.csv text files), extract settings and initialize the dictionaries of network objects as well as the computation grid:

In [ ]:
# Path to this notebook
home = os.getcwd() 

# Locations of csv configuration files
path_struct_img = home + '/Parameter_Files/structured_image_dir.csv'
path_multilayer = home + '/Parameter_Files/multilayer_dir_parameters.csv'
path_startpar = home + '/Parameter_Files/starting_positions.csv'
path_growth = home + '/Parameter_Files/growth_parameters.csv'
path_internal = home + '/Parameter_Files/internal_parameters.csv'

# Generate feature maps from image data of growth environment
features = ing.StructField_Maker.structured_field(path_struct_img, home, sigma_divd=2,
                                                  sigma_divt1=2, sigma_divt2=2)

# Initialise the computation grid
field_ref = ing.StructField_Maker.interpol_external_fields(path_multilayer)

# Initialize the object dictionaries
obj_par_env = ing.init_objects_parameter_enviroment(path_startpar,path_internal,path_growth,path_multilayer)

The growth process is implemented as parallel simulation, therefore the data need to be transferred into shared memory objects, using the [**manager concept**](https://docs.python.org/3/library/multiprocessing.html#managers) from the python [**multiprocessing**](https://docs.python.org/3/library/multiprocessing.html) library:

In [ ]:
# Extract individual dictionaries
cod, nod, edd, flavour, ed_names, no_names, co_names, field, steps, instances, dim, radius  = obj_par_env

# Create shared memory proxy objects for all paramter and Class objects
mgr, eddp, nodp, flavourp, ed_namesp, \
no_namesp, co_namesp, fieldp, forcep = ing.init_manager_init(edd, nod, flavour,
                                                            ed_names, no_names,
                                                            co_names, field,
                                                            field_ref)

# Transfer proxy objects to the correct namespace (needed when running in notebook)
ing.mgr = mgr
ing.eddp = eddp
ing.nodp = nodp
ing.flavourp = flavourp
ing.ed_namesp = ed_namesp
ing.no_namesp = no_namesp
ing.co_namesp = co_namesp
ing.fieldp = fieldp
ing.forcep = forcep

Finally, set the number of instances (simulation steps) and substeps per instance, and run the parallel simulation: 

In [ ]:
#Starting the growth process simulation

%matplotlib widget

instances = 5
steps = 10

growing_results = ing.init_par_growing(cod, nod, edd, steps, 
                                  instances, dim, radius,
                                  eddp, nodp, flavourp, fieldp,
                                  ed_namesp, no_namesp, co_namesp,
                                  forcep,
                                  timer = True,
                                  plot = True,
                                  Voronoi = False,
                                  network_x = True)

cod, nod, edd, flavour, ed_names, no_names, co_names, vor, G_vor, subG_vor, G, subG = growing_results